# Federated Learning

The Federated Emotion Analysis Model is a cutting-edge approach to emotion detection that leverages federated learning techniques. Unlike traditional centralized models, federated learning allows for the training and updating of machine learning models directly on user devices, ensuring privacy, continuous learning, and user engagement.

## Advantages and Feasibility:

1. **Privacy Benefits:** By training the model directly on user devices, sensitive user data remains on the device and is not sent to a central server. This ensures user privacy and data security, making it an attractive option for applications handling sensitive information.
2. **Continuous Learning:** The model continuously learns and adapts based on user interactions and feedback. This ensures that the model stays relevant and up-to-date with evolving user behavior and language patterns.
3. **User Interaction and Feedback Mechanism:** Users actively participate in the model training process by providing feedback on the detected emotions. This fosters user engagement and creates a more personalized experience.
4. **Localized Data Processing:** By conducting model training and updates on user devices, the need to transfer sensitive data to a central server is eliminated. This localized processing minimizes data exposure and reduces the risk of unauthorized access or breaches.
5. **Reduced Communication Overhead:** Federated learning distributes model updates across multiple devices, reducing the burden on centralized infrastructure and mitigating communication overhead. This results in improved scalability and efficiency, particularly in resource-constrained environments.
6. **Customization and Personalization:** The decentralized nature of federated learning allows for tailored model updates based on individual user experiences and preferences. This enables the delivery of personalized emotion detection capabilities that better align with diverse user needs and contexts.

# EmoTrack Implementation: Protype for Federated learning 

In [1]:
pip install --quiet --upgrade tensorflow-federated

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
trie 2.0.0a5 requires typing-extensions<4,>=3.7.4, but you have typing-extensions 4.5.0 which is incompatible.
mythx-models 2.2.0 requires jsonschema==3.2.0, but you have jsonschema 4.19.2 which is incompatible.
mythx-models 2.2.0 requires python-dateutil==2.8.1, but you have python-dateutil 2.9.0.post0 which is incompatible.
pythx 1.7.3 requires PyJWT==1.7.1, but you have pyjwt 2.8.0 which is incompatible.
pythx 1.7.3 requires python-dateutil==2.8.1, but you have python-dateutil 2.9.0.post0 which is incompatible.
pythx 1.7.3 requires requests==2.25.1, but you have requests 2.31.0 which is incompatible.
mypy 1.8.0 requires mypy-extensions>=1.0.0, but you have mypy-extensions 0.4.4 which is incompatible.
conda 24.1.2 requires packaging>=23.0, but you hav

In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

import sqlite3
import pickle
import os

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('user_model.db')
cursor = conn.cursor()

## Initial Model Distribution

* Load the trained global model from the .pkl file.
* Distribute this model to the client devices upon installation of the app.

In [ ]:
# Function to load the global model from .pkl file
def load_global_model(pkl_file_path):
    with open(pkl_file_path, 'rb') as file:
        global_model = pickle.load(file)
    return global_model

# Function to distribute the global model to client devices
def distribute_model_to_clients(global_model, client_devices):
    for device_id in client_devices:
        # Example: Save the model to each client's directory
        client_model_path = f'client_{device_id}_model.pkl'
        with open(client_model_path, 'wb') as file:
            pickle.dump(global_model, file)
        print(f"Global model distributed to client {device_id}")

# Function to create SQLite database table for user emotion analysis
def create_user_model_table(database_path, table_name):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    cursor.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (
                        user_id INTEGER PRIMARY KEY,
                        user_text TEXT,
                        detected_emotion TEXT
                    )''')
    conn.commit()
    conn.close()
    print(f"Table '{table_name}' created in the SQLite database.")

# Main function
def main():
    # Path to the global model .pkl file
    global_model_pkl_path = 'emotion_classifier_global_model.pkl'
    
    # Load global model
    global_model = load_global_model(global_model_pkl_path)
    
    # List of client devices (for demonstration)
    client_devices = [1, 2, 3]  # Replace with actual client device IDs
    
    # Distribute global model to client devices
    distribute_model_to_clients(global_model, client_devices)
    
    # Path to the SQLite database file
    database_path = 'user_model.db'
    
    # Create table in SQLite database for user emotion analysis
    for device_id in client_devices:
        table_name = 'user_model'+device_id
        create_user_model_table(database_path, table_name)

if __name__ == "__main__":
    main()

## Server Implementation

In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('user_model.db')
cursor = conn.cursor()

# Function to aggregate user feedback and update global model
def update_global_model():
    # Query user feedback from database
    cursor.execute("SELECT * FROM user_model")
    feedback_data = cursor.fetchall()
    
    # Aggregate feedback and update global model
    aggregated_feedback = process_feedback(feedback_data)
    global_model.update(aggregated_feedback)
    
    # Save updated global model to .pkl file
    save_updated_model(global_model)

# Function to save updated global model
def save_updated_model(model):
    # Save model to .pkl file
    with open('updated_global_model.pkl', 'wb') as file:
        pickle.dump(model, file)

# Function to process and aggregate user feedback
def process_feedback(feedback_data):
    # Process feedback data to extract relevant information
    # Aggregate feedback to update global model
    aggregated_feedback = ...
    return aggregated_feedback

# Call update_global_model periodically
while True:
    # Update global model every 'n' days (adjust as needed)
    time.sleep(n_days)
    update_global_model()


## Client Implementation

In [ ]:
import pickle

# Load global model from .pkl file
with open('emotion_classifier_pipe_lr.pkl', 'rb') as file:
    global_model = pickle.load(file)

# Function to detect emotions from user text
def detect_emotion(text):
    # Use global model to predict emotion from text
    emotion = global_model.predict(text)
    return emotion

# Function to receive user feedback and update local database
def receive_feedback(user_id, feedback):
    # Save user feedback to local database (e.g., SQLite)
    save_feedback_to_database(user_id, feedback)

# Main loop for user interaction
while True:
    # Get user input (text message)
    text_message = input("Enter your text message: ")
    
    # Detect emotion from user text
    emotion = detect_emotion(text_message)
    
    # Provide detected emotion to user (optional)
    print("Detected emotion:", emotion)
    
    # Receive user feedback
    feedback = input("Provide feedback on the detected emotion (positive/negative): ")
    
    # Save user feedback to local database
    receive_feedback(user_id, feedback)


## Refernces

1. https://www.tensorflow.org/federated/tutorials/building_your_own_federated_learning_algorithm
2. https://github.com/microsoft/personalizedfl
3. V. Tsouvalas, T. Ozcelebi and N. Meratnia, "Privacy-preserving Speech Emotion Recognition through Semi-Supervised Federated Learning", doi: 10.1109/PerComWorkshops53856.2022.9767445.